In [1]:
import os
import numpy as np
import cv2
import keras.utils 
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import math
import pandas as pd

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def read_data_info_from_csv(csv_path,save_path,is_crop=False):
    """
    #Argument:
        csv_path: path of file csv that container information of data per class 
        (ex: file name, size, roi.x, roi.y, class id)
    #Return:
        images, labels, number image of class
        type: list
    """
    print(csv_path)
    images =[]
    labels=[]
    data_info_csv = pd.read_csv(csv_path, sep=';')
    num_img_of_class = len(data_info_csv["Filename"])
    parent_dir_name =os.path.dirname(csv_path)
    
    for i in range(num_img_of_class):
        #read information of csv file
#         0000_000.ppm
        img_name = data_info_csv["Filename"][i]
#         0000_000
        img_name_without_extension = img_name.split('.')[0]
#         print(img_name_without_extension)
        img_suffix=".png"
        img_save_path = os.path.join(save_path,img_name_without_extension+img_suffix)
#         print(img_save_path)

        roi_x1=data_info_csv['Roi.X1'][i]
        roi_y1 = data_info_csv['Roi.Y1'][i]
        roi_x2 =data_info_csv['Roi.X2'][i]
        roi_y2 =data_info_csv['Roi.Y2'][i]
        label = data_info_csv['ClassId'][i]
        img_path = os.path.join(parent_dir_name,img_name)
        image = cv2.imread(img_path)
        cv2.imwrite(img_save_path,image)
        if is_crop:
             image = image[roi_y1:roi_y2,roi_x1:roi_x2,:]
        image = cv2.resize(image,(32,32))
        images.append(image)
        labels.append(label)
    
    return  images,labels,num_img_of_class

def read_data(data_dir,is_crop=False):
    """
    #argument:
        data_dir: path of directory
    #return:
        images, labels, number of sample
    """
     # Get all subdirectories of data_dir. Each represents a label.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
     # Loop through the label directories and collect the data in
    # two lists, labels and images.
    print(f"directories: {directories}")
    labels = []
    images = []
    num_sample =0;
    num_classes =0;
    total_num_img =0
    for index,d in enumerate(directories):
        label_dir = os.path.join(data_dir, d)
#         print(os.path.dirname(os.getcwd()))
        save_path =os.path.join(os.path.dirname(os.getcwd()),"imagedata",d)
        print(save_path)

        if not os.path.isdir(save_path):
            os.makedirs(save_path)
        #find path data information in csv file   
        path_info_data_csv = [os.path.join(label_dir, f)  for f in os.listdir(label_dir) if f.endswith(".csv")]
        
        for fn_csv in path_info_data_csv:      
            imgs, lbs,num_img_of_class = read_data_info_from_csv(fn_csv,save_path,is_crop)
            total_num_img = total_num_img+num_img_of_class;
    
            images.extend(imgs)
            labels.extend(lbs)
            
            print(f"index_name: {index}, directory : {d}, total number of image: {num_img_of_class}")
        num_classes=num_classes+1; 
    print(f"total_num_img: {total_num_img}")
    
    #read label's name
 

    images= np.asarray(images)
    labels = np.asarray(labels)
    
    return images, labels,num_classes



In [ ]:
data_train_dir=r"..\data\train\GTSRB"
_,_,_ = read_data(data_train_dir,is_crop=False)

In [85]:
import pickle  
def read_data_enhancement(file_name):
    """
    #argument: 
        file_name: file's name is loaded
    #return: 
        images, labels
    """
    pickle_fn = os.path.join(r"..\data_enhancement",file_name)
    if os.path.isfile(pickle_fn):
        data= pickle.load(open(pickle_fn,'rb'))
        print(f'done read_dataa from {file_name}')
        return data
    else:
        raise Exception("the file data enhancement is not found")
data = read_data_enhancement('primitive\data_30_09.p')

done read_dataa from primitive\data_30_09.p


In [88]:
type(data)
data.keys()
lbs = data['lbs']
print(np.max(data['ims']))

1.0


In [97]:
def save_img(file_path_read):
    data = read_data_enhancement(file_path_read)
    lbs =data['lbs']
   
    ims = data['ims']
    print(lbs.shape)
    print(ims.shape)
    save_path_base = os.path.join(os.path.dirname(os.getcwd()),"image\\full")
    if not os.path.isdir(save_path_base):
        os.makedirs(save_path_base)
    for i, lb in enumerate(lbs):
        save_path = os.path.join(save_path_base,str(lb))
        if not os.path.isdir(save_path):
            os.makedirs(save_path)
        img_save_path = os.path.join(save_path,str(i)+".png")
        cv2.imwrite(img_save_path,ims[i]*255.)

In [98]:
save_img('full\data_3009.p')

done read_dataa from full\data_3009.p
(146545,)
(146545, 32, 32, 3)


In [7]:
"""image test"""

_,_,_ = read_data_info_from_csv(r"..\data\test\GTSRB\GT_final_test.csv",r"..\image\test")

..\data\test\GTSRB\GT_final_test.csv


In [ ]:
def save_image_test(csv_path,save_path,is_crop=False):
    """
    #Argument:
        csv_path: path of file csv that container information of data per class 
        (ex: file name, size, roi.x, roi.y, class id)
    #Return:
        images, labels, number image of class
        type: list
    """
    print(csv_path)
    data_info_csv = pd.read_csv(csv_path, sep=';')
    num_img_of_class = len(data_info_csv["Filename"])
    parent_dir_name =os.path.dirname(csv_path)
    
    for i in range(num_img_of_class):
        img_name = data_info_csv["Filename"][i]
#         0000_000
       
#         print(img_save_path)

        roi_x1=data_info_csv['Roi.X1'][i]
        roi_y1 = data_info_csv['Roi.Y1'][i]
        roi_x2 =data_info_csv['Roi.X2'][i]
        roi_y2 =data_info_csv['Roi.Y2'][i]
        label = data_info_csv['ClassId'][i]
       
        img_name_without_extension = img_name.split('.')[0]
        img_suffix=".png"
        save_path_base = os.path.join(save_path,str(label))
        if not os.path.isdir(save_path_base):
            os.makedirs(save_path_base)
        img_save_path = os.path.join(save_path_base,img_name_without_extension+img_suffix)
        img_path = os.path.join(parent_dir_name,img_name)
        image = cv2.imread(img_path)
        cv2.imwrite(img_save_path,image)

save_image_test(r"..\data\test\GTSRB\GT_final_test.csv",r"..\image\test_v2")